__применение парсера объявлений циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-09-13 13:49:38 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-13 13:49:38 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-13 13:49:40 | AdsListParser: downloader init
[INFO    ] 2022-09-13 13:49:40 | CianParser: downloader init
[INFO    ] 2022-09-13 13:49:40 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-13 13:49:50 | AdsListParser: read page 1
[INFO    ] 2022-09-13 13:50:01 | AdsListParser: read page 2
[INFO    ] 2022-09-13 13:50:14 | AdsListParser: read page 3
[INFO    ] 2022-09-13 13:50:23 | AdsListParser: read page 4
[INFO    ] 2022-09-13 13:50:39 | AdsListParser: read page 5
[INFO    ] 2022-09-13 13:50:50 | AdsListParser: read page 6
[INFO    ] 2022-09-13 13:50:56 | AdsListParser: read page 7
[INFO    ] 2022-09-13 13:51:05 | AdsListParser: read page 8
[INFO    ] 2022-09-13 13:51:17 | AdsListParser: read page 9
[INFO    ] 2022-09-13 13:51:31 | AdsListParser: read page 10
[INFO    ] 2022-09-13 13:51:38 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
122,"2-комн. кв., 145 м², 6/7 этаж",,,29 000 000 ₽,200 000 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Стрелецкая бухта, улица Дмитрия Ульянова, 11А]","[5 часов назад, сегодня, 08:41]",https://sevastopol.cian.ru/sale/flat/269048917/,"Двухкомнатная квартира 145 м2 на 6/7 этаже дома с большой кухней 35м2. Расположена в г. Севастополь, Гагаринский район, ул. Дмитрия Ульянова, д.11А. Элитный дом, с подземным паркингом (у нас личный гараж - 50м, приобретение обсуждается с реальным покупателем) из которого лифт доставит вас до квартиры.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Гагаринский район, ул. Дмитрия Ульянова, д.11А.\n-Год постройки дома: 2013 г.\n-Количество комнат: 2\n-Комнаты: изолированы\n-Общая площадь: 145 м2\n-Жилая площадь: 80 м2\n-Этаж /этажность: 6/7\n-Площадь кухни: 35 м2\n-Высота потолка: 3 м\n-Санузел: два совмещенных\n-Водоснабжение: центральное\n-Горячее водоснабжение: АГВ\n-Отопление: АГВ\nДополнительная информация: \nКвартира в сегменте премиум класса, в лучшем Гагаринском районе города Севастополя Стрелецкой Бухте. Общая площадь квартиры составляет 145 м². В квартире выполнен качественный дизайнерский ремонт. Барная стойка, столешница на кухне, раковины в с/у из искусственного камня. В комнатах на полу массив дерева под лаком (Германия). Все остальные полы мрамор (Турция) - тёплый пол. Подоконники - мрамор. На кухне: встроенные духовка, микроволновка, холодильник, посудомойка, плита газовая электролюкс, газовый котёл Бакси. Кондиционеры - Хаер, Дайкин, Грии. Телевизоры 55""изогнутый. Кровать заказная, матрац ортопедический. Шторы на заказ. \nПозвоните нам, чтобы назначить просмотр! \nИнфраструктура: \nРядом с парком имени Юрия Гагарина, а также со всей необходимой инфраструктурой для комфортной жизни.\n На первом этаже расположен ""Московский Центр косметологии"" и студия красоты ""Zazeркалье"". В шаговой доступности расположена школа 34 детские сады 88 и 20, поликлиники, почтовое отделение, управляющая компания, продуктовые магазины, пляжи ""Солнечный"" и ""Песочный"", а также остановки общественного транспорта, откуда вы сможете удобно добраться в любой район города.\nВ 5-и минутах ходьбы находится одна из центральных достопримечательностей нашего города - музей-заповедник ""Херсонес Таврический"".\n Подходит для наличного расчета, под гражданскую ипотеку, материнский капитал, жилищные сертификаты. \nДокументы РФ, проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять! \nВозможно произошёл сбой и Вы не смогли дозвониться. Напишите Ваш номер, мы свяжемся с Вами в ближайшее время или попробуйте позвонить ещё раз.",5,2022-09-13 13:59:12.511147
99,"2-комн. кв., 61,5 м², 3/8 этаж",Сдача корпуса 1 кв. 2024,сдача ГК: 1 кв. 2024 года,13 653 000 ₽,222 000 ₽/м²,"[Севастополь, р-н Гагаринский, мкр. Летчики]","[9 часов назад, сегодня, 04:11]",https://sevastopol.cian.ru/sale/flat/270388687/,"Продаётся 2-комнатная квартира в строящемся доме (Секция 1), срок сдачи: I-кв. 2024, общей площадью 61.5 кв.м., на 3 этаже. Квартира в жилом комплексе Дом на Маячной"" - это ваша возможность не выбирать между морем и активной жизнью, а брать все лучшее сразу. \nЖилой комплекс ""Дом на Маячной"" имеет прекрасное расположение, где близость моря сочетается с развитой инфраструктурой района. Каждый день вы будете дышать морским воздухом, находясь в самом эпицентре событий. \nИнновационный подход к архитектуре и строительству \nИспользование только современных и качественных материаловИзысканные архитектурные решенияДизайнерские, эргономичные и вентилируемые фасады, которые отлично вписываются в прибрежный ландшафтУдобные планировки: это просторные кухни и гостиные, тихие спальни, большие окнаДля сохранения пространства квартиры предусмотрены отдельные кладовыеДизайнерские холлыД

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1509
1506


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [9]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [10]:
# df

In [11]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [12]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [13]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])